In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation,BatchNormalization,Lambda,Input,Conv2D,Bidirectional,LSTM, Concatenate, MaxPooling2D,GlobalAveragePooling2D
import tensorflow.keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import time
import datetime
import numpy as np
from generater import crnnGenerator
import logging
logger=tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:

def convert_to_onehot(data):
    #Creates a dict, that maps to every char of alphabet an unique int based on position
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    
    encoded_data = []
    #Replaces every char in data with the mapped int
    encoded_data.append([char_to_int[char] for char in data])
    
    

    #This part now replaces the int by an one-hot array with size alphabet
    one_hot = []
    for value in encoded_data:
        #At first, the whole array is initialized with 0
        
        for indexvalue in value:
            letter = [0 for _ in range(len(alphabet))]
            #Only at the number of the int, 1 is written

            letter[indexvalue] = 1
            
            one_hot.append(letter)
    return one_hot

def convert_to_lable(data):
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    return list(map(lambda x: alphabet.index(x), data))



convert_to_lable('abcd453')


[0, 1, 2, 3, 30, 31, 29]

In [6]:

def crnn():
    x=image_input=Input(name='image_input',shape=[128,64,1],dtype='float32')
    """
    minx=Input(name='minx',shape=[1],dtype='float32')
    miny=Input(name='miny',shape=[1],dtype='float32')
    maxx=Input(name='maxx',shape=[1],dtype='float32')
    maxy=Input(name='maxy',shape=[1],dtype='float32')
    def cropimage(img,minx=0,miny=0,maxx=0,maxy=0):
        
        return tf.image.crop_to_bounding_box(img,minx,miny,maxx-minx,maxy-miny)
    x=Lambda(cropimage)([x,minx,miny,maxx,maxy])
    
    """
        
    x=Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_1')(x)
    x=MaxPooling2D(pool_size=(2, 2), name='pool1', padding='same')(x)#64 32
    
    x=Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv2_1')(x)
    #x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2', padding='same')(x)# 32 16

    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_1')(x)
    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_2')(x)
    #x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool3', padding='same')(x)# 16 8
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_1')(x)
    x=BatchNormalization(name='batch1')(x)
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_1')(x)
    
    x=BatchNormalization(name='batch2')(x)
    #x=MaxPooling2D(pool_size=(2, 2), strides=(1, 2), name='pool5', padding='valid')(x)# 8 4
    #63 31 512
    x=aa=Conv2D(512, (2, 2), strides=(1, 1), activation='relu', padding='valid', name='conv6_1')(x)#63 31 512
    x=keras.layers.Reshape((-1,512))(x)
    print(aa)
    x=Bidirectional(LSTM(256, return_sequences=True))(x)
    x=Bidirectional(LSTM(256, return_sequences=True))(x)
    
    num_classes=36+1
    x=keras.layers.Dense(num_classes, name='dense1')(x)#알파벳+숫자#1953 36
    
    x=y_pred= Activation('softmax', name='softmax')(x)#1953 36
    
   # model_pred=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy],outputs=x)
    model_pred=keras.models.Model(inputs=image_input,outputs=x)
    
    #maxstringlen=int(y_pred.shape[1])
    maxstringlen=100
    
    def ctc_lambda_func(args):
        y_pred,labels, input_length, label_length = args
          #y_pred = y_pred[:, 2:, :] 
        return K.ctc_batch_cost(y_true=labels,y_pred=y_pred,input_length=input_length,label_length=label_length)    
        #return tf.nn.ctc_loss(labels=labels,logits=y_pred,logit_length=input_length,label_length=label_length,logits_time_major=False)    
    
    labels = Input(name='labels', shape=[maxstringlen], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64') 
    
    ctcloss=Lambda(ctc_lambda_func,output_shape=(1,),name='ctcloss')([y_pred,labels,input_length,label_length])
    
   # model_train=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy,labels,input_length,label_length],outputs=ctc_loss)
    model_train=keras.models.Model(inputs=[image_input,labels,input_length,label_length],outputs=ctcloss)
    
    return model_train,model_pred,maxstringlen

    

In [7]:
traincrnn,predcrnn,inputlength=crnn()

KerasTensor(type_spec=TensorSpec(shape=(None, 63, 31, 512), dtype=tf.float32, name=None), name='conv6_1/Relu:0', description="created by layer 'conv6_1'")


In [8]:
cropvalpath='D:/ocr/croptestimage/'
croptrainpath='D:/ocr/croptestvalimage/'
cropvaljson='D:/ocr/valcrnn.json'
batch=32
traingen=crnnGenerator(imgpath=croptrainpath,labelpath=cropvaljson,imgw=128,imgh=64,batch_size=batch,
                      maxlen=inputlength,inputlen=inputlength)
print('trainstart')
traingen.build_data()
print('valstart')
valgen=crnnGenerator(imgpath=cropvalpath,labelpath=cropvaljson,imgw=128,imgh=64,batch_size=batch,
                      maxlen=inputlength,inputlen=inputlength)
valgen.build_data()
print('valend')

trainstart
valstart
valend


In [9]:
"""
import winsound as sd

def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()
"""
inputlength

100

In [10]:
traincrnn.compile(optimizer='adam',loss={'ctcloss':lambda labels,y_pred:y_pred})

In [11]:
from tensorflow.python.keras.callbacks import TensorBoard  ## TensorBoard 를 import합니다.
modelver='crnn_train_v1'
checkdir = './checkpoints/' + datetime.datetime.now().strftime('%Y%m%d%H%M') + '_' + modelver
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())
log_dir = "./logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard =TensorBoard(log_dir=log_dir,histogram_freq=1)

In [12]:
#[image_input,labels,input_length,label_length]
"""
his=traincrnn.fit(x=[valdata,valdata.labels,np.array( inputlengthlist),np.array(vallabellength)],
              y=valdata.labels,
                batch_size=16, 
              epochs=15,
              callbacks=[keras.callbacks.ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, save_weights_only=True),
                         tensorboard]
             )
"""
his=traincrnn.fit_generator(generator=traingen.next_batch(),
                            steps_per_epoch=int(traingen.imgcount/batch),
                            epochs=10,
                            callbacks=[keras.callbacks.ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, moniter='loss'),
                            tensorboard],
                            validation_data=valgen.next_batch(),
                            validation_steps=int(valgen.imgcount/batch)
                            
                           
                           
                           )



c:\users\user\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
451/451 [==============================] - 695s 2s/step - loss: 17.0959 - val_loss: 16.3440

Epoch 00001: saving model to ./checkpoints/202107180819_crnn_train_v1\weights.001.h5
Epoch 2/10
451/451 [==============================] - 678s 2s/step - loss: 15.8698 - val_loss: 15.7036

Epoch 00002: saving model to ./checkpoints/202107180819_crnn_train_v1\weights.002.h5
Epoch 3/10
451/451 [==============================] - 681s 2s/step - loss: 15.5599 - val_loss: 15.7733

Epoch 00003: saving model to ./checkpoints/202107180819_crnn_train_v1\weights.003.h5
Epoch 4/10
451/451 [==============================] - 681s 2s/step - loss: 15.2168 - val_loss: 15.3377

Epoch 00004: saving model to ./checkpoints/202107180819_crnn_train_v1\weights.004.h5
Epoch 5/10
451/451 [==============================] - 680s 2s/step - loss: 15.0364 - val_loss: 14.9878

Epoch 00005: saving model to ./checkpoints/202107180819_crnn_train_v1\weights.005.h5
Epoch 6/10
451/451 [==============================] - 6

tf.Tensor(
[[[[ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   ...
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]]

  [[ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   ...
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]]

  [[ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   [ 54.         53.         48.       ]
   ...
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]
   [ 78.        100.         28.       ]]

  ...

  [[ 30.         30.         42.       ]
   [ 30.         30.         42.       ]
   [ 30.         30.         42.       ]
   ...
   [ 63.         62.         42.       ]
   [ 63.         62.         42.       ]
   [ 

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,


8001

[[28, 27, 28, 28, 26, 32, 27, 35, 34, 31],
 [17, 4, 3],
 [2, 14, 17, 13, 4, 17],
 [28, 28, 26],
 [2, 14, 17, 13, 4, 17],
 [17, 4, 3],
 [2, 0, 5, 4],
 [0, 19, 12],
 [28, 30],
 [7, 14, 20, 17],
 [0, 19, 12],
 [0],
 [19],
 [12],
 [18, 0, 11, 0, 3, 18],
 [2, 0, 5, 4],
 [2, 14, 17, 13, 4, 17],
 [17, 4, 3],
 [22, 14, 10],
 [4],
 [18, 0, 13, 3, 12, 8, 2, 7, 4, 18],
 [5, 14, 14, 3],
 [19, 0, 10, 4],
 [14, 20, 19],
 [1, 1],
 [2, 14, 5, 5, 4, 4],
 [1, 0, 17],
 [10, 4, 1, 0, 1, 18],
 [18, 12, 14, 14, 19, 7, 8, 4, 18],
 [2, 0, 5, 4],
 [8, 1, 12],
 [8, 1, 12],
 [13, 14, 22],
 [15, 11, 0, 24, 8, 13, 6],
 [8, 13],
 [19, 7, 4],
 [22, 8, 13, 13, 4, 17, 18],
 [2, 11, 14, 20, 3],
 [27, 27, 28, 26, 27, 26],
 [30, 28, 26, 27, 26],
 [27, 26, 28, 26, 26, 35],
 [17, 4, 18, 4, 0, 17, 2, 7],
 [31, 28, 26, 26, 35],
 [27, 28, 28, 26, 26, 34],
 [34, 28, 26, 26, 34],
 [31, 28, 26, 26, 34],
 [27, 26, 26],
 [32, 28, 26, 26, 33],
 [27, 27, 27, 28, 26, 27, 26],
 [35, 26],
 [34, 26],
 [33, 26],
 [3, 4, 4, 15, 16, 0],
 [